# Grid de hiperparámetros

### Instalar la versión de scikit-learn compatible con scikeras

In [ ]:
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.5.2

Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 110.2 MB/s eta 0:00:00


### Instalar el paquete  *scikeras* si no está disponible

In [ ]:
try:
    import scikeras
except ImportError:
    print("scikeras no preinstalado en Colab:\n\n")
    !python -m pip install scikeras



### Importaciones

In [ ]:
import numpy as np
from keras.datasets import mnist
from keras.layers import Dense, Input, Activation, Dropout
from keras.models import Sequential
from time import time
from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.utils import to_categorical
from sklearn.model_selection import GridSearchCV

### Carga del dataset en arrays de Numpy
Información del dataset MNIST:  [[wikipedia](https://en.wikipedia.org/wiki/MNIST_database)]

In [ ]:
(X_entr, y_entr), (X_val, y_val) = mnist.load_data()

### Preparación de los datos

In [ ]:
# clases distintas para identificar (10 dígitos)
num_clases = 10

# cambio de dimensiones de las imágenes (28x28 -> 784)
X_entr = X_entr.reshape(X_entr.shape[0], 784)
X_val  = X_val.reshape(X_val.shape[0],   784)

# conversión de enteros a float32
X_entr = X_entr.astype('float32')
X_val  = X_val.astype('float32')

# Normalización
X_entr /= 255
X_val  /= 255

# codificación 'one-hot'
Y_entr = to_categorical(y_entr, num_clases)
Y_val  = to_categorical(y_val, num_clases)

print(Y_entr[0:3, :])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


### Creación y compilación del modelo

In [ ]:
def creaModelo(inicial):
    modelo = Sequential()
    modelo.add(Input(shape=(784,)))

    modelo.add(Dense(512, kernel_initializer=inicial, activation='relu'))
    modelo.add(Dropout(0.2))
    modelo.add(Dense(512, kernel_initializer=inicial, activation='relu'))
    modelo.add(Dropout(0.2))
    modelo.add(Dense(10, kernel_initializer=inicial, activation='softmax'))

    modelo.compile(loss='categorical_crossentropy', metrics=['accuracy'])

    return modelo


### Configuración de los hiperparámetros que se probarán

In [ ]:
lotes           = [16, 32]
optimizadores   = ['rmsprop', 'adam']
inicializadores = ['glorot_uniform', 'normal', 'uniform']
epochs          = [2, 3]

# parámetros a probar: argumentos de la función que crea el modelo y los de fit
param_grid = {'optimizer': optimizadores, 'model__inicial':inicializadores, 'epochs':epochs, 'batch_size':lotes}

### Envoltorio clasificador: recibe la función que crea y compila la red


In [ ]:
modelo = KerasClassifier(model=creaModelo)

### Construcción del grid de pruebas e inicio del entrenamiento

In [ ]:
# explorador de combinaciones
grid = GridSearchCV(estimator=modelo, param_grid=param_grid)

# control del tiempo de ejecución
inicio=time()

# entrenamiento sobre el GridSearchCV
resultado_grid = grid.fit(X_entr, Y_entr)
print("Tiempo total:",time()-inicio)

Epoch 1/2
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - accuracy: 0.8842 - loss: 0.3774
Epoch 2/2
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9644 - loss: 0.1412
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1/2
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8843 - loss: 0.3740
Epoch 2/2
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9631 - loss: 0.1399
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1/2
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8909 - loss: 0.3614
Epoch 2/2
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9654 - loss: 0.1361
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1/2
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8895 - loss: 0.3687
Epoch 2/2
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9644 - loss: 0.1360
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Epoch 1/2
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8855 - loss: 0.3781
Epoch 2/2
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step

/usr/local/lib/python3.11/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Epoch 1/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8872 - loss: 0.3625
Epoch 2/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9666 - loss: 0.1130
Epoch 3/3
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9756 - loss: 0.0887
Tiempo total: 2440.7962856292725


### Acceso a la mejor combinación de hiperparámetros

In [ ]:
print("\n\nMejor resultado:", resultado_grid.best_score_,"usando", resultado_grid.best_params_,"\n")
# la mejor combinación también está en grid.best_params_

medias = resultado_grid.cv_results_['mean_test_score']
desvs  = resultado_grid.cv_results_['std_test_score']
params = resultado_grid.cv_results_['params']

for i in range(len(medias)):
  print("media:",medias[i], "(desv: ", str(desvs[i])+") ->", params[i])



Mejor resultado: 0.9744166666666667 usando {'batch_size': 32, 'epochs': 3, 'model__inicial': 'glorot_uniform', 'optimizer': 'rmsprop'} 

media: 0.9674833333333334 (desv:  0.0037369773882109355) -> {'batch_size': 16, 'epochs': 2, 'model__inicial': 'glorot_uniform', 'optimizer': 'rmsprop'}
media: 0.9661333333333333 (desv:  0.004062908099160277) -> {'batch_size': 16, 'epochs': 2, 'model__inicial': 'glorot_uniform', 'optimizer': 'adam'}
media: 0.9693166666666666 (desv:  0.002182887995294315) -> {'batch_size': 16, 'epochs': 2, 'model__inicial': 'normal', 'optimizer': 'rmsprop'}
media: 0.9687666666666667 (desv:  0.0030447587023532097) -> {'batch_size': 16, 'epochs': 2, 'model__inicial': 'normal', 'optimizer': 'adam'}
media: 0.9687000000000001 (desv:  0.001918187802183227) -> {'batch_size': 16, 'epochs': 2, 'model__inicial': 'uniform', 'optimizer': 'rmsprop'}
media: 0.9682333333333334 (desv:  0.0036014657509902247) -> {'batch_size': 16, 'epochs': 2, 'model__inicial': 'uniform', 'optimizer':